# 🎨 Manipulace s obrazovými daty

V této ukázce se budeme věnovat obrazovým datům a základním operacím s nimi. 

🗒️ Poznámka: V tomto notebooku používáme interaktivní prvky. Pro jejich správné zobrazení je třeba notebook spustit. Například pomocí `Cell > Run all Cells `.

Také se ujistěte, že máte nainstalovaný modul `ipywidgets` a je povolený v `jupyteru`. Pokud ne, lze to napravit následujícími kroky:

- nainstalovat ipywidgets (nemáte-li)
    > conda install -c conda-forge ipywidgets
- povolit ipywidgets
    > jupyter nbextension enable --py widgetsnbextension
- restartovat jupyter notebook (běží-li)

Obrázky pro toto cvičení pochází z [thecatapi](https://api.thecatapi.com).

⛔️ **POZOR!** Pro toto cvičení nedoporučujeme používat prohlížeč Safari 🦒 kvůli chybnému rozvržení interaktivních oken.

In [ ]:
import matplotlib.pyplot as plt
from PIL import ImageOps
import cv2 as cv
import sys

import io
import mpmath

sys.modules["sympy.mpmath"] = mpmath

import skimage
import glob

import helpers
import sympy
import numpy as np

from IPython.display import Latex, Math
from ipywidgets import widgets

import scipy.spatial.distance as dist
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering
from PIL import Image, ImageEnhance, ImageStat, ImageFilter, ImageTransform

from sklearn.cluster import KMeans

# Pillow<9.0import skimage
if not hasattr(Image, "Resampling"):
    Image.Resampling = Image

Ukážeme si několik modulů pro práci s obrazovými daty. Pro tyto potřeby si pomocí modulu [`PIL`](https://pillow.readthedocs.io/en/stable/) načteme obrázek kočky 🐈, na kterém budeme vše demonstrovat.

In [ ]:
cat = Image.open("selected_cat.png")
cat

`Image` objekt z modulu [`PIL.Image`](https://pillow.readthedocs.io/en/stable/reference/Image.html?highlight=image) neoplývá mnoha užitečnými atributy (vesměs se potkáte primárně s `Image.size`, který se skládá z `Image.width` a `Image.height`) a jen s malým výčtem metod. Hlavní funkckionality se schovávají v dalších submodulech balíčku `PIL` a postupně si je představíme. 🧑🏽‍🏫

In [ ]:
# cat.size == (cat.width, cat.height)
cat.size

Pokud byste používali `scikit-image` a atribut `shape`, budou vaše data trojsložková: první dva údaje vypovídají o velikosti x a y-ové osy, třetí pak udává barevnost. `shape` je atribut definovaný pro `numpy.ndarray` (`PIL.Image` nepracuje s obrázky jako s poli). 

Ukažme si výstup atributu `shape` na příkladu.

In [ ]:
chelsea = skimage.data.chelsea()
fig, ax = plt.subplots()
ax.imshow(chelsea)
print('Tvar obrázku:', chelsea.shape)

Pokud trváte na získání barevnosti i pro `PIL.Image`, pak můžete využít:

In [ ]:
np.shape(cat)

## ☝️ Histogram

Z přednášky víte, že jedním ze zdrojů informace o obrázku je `histogram` barev. 📊 Instance obrázků mají metodu `histogram()`, která vrátí pole hodnot odpovídající četnostem barev jednotlivých pixelů. 

U `grayscale` obrázku bychom dostali pole o velikosti `256`, u `RGB` obrázku dostaneme pole o `3x256` hodnotách. 

Histogram barev lze vizualizovat. Z grafu níže můžeme vyčíst, jaké hodnoty byly nejčastější, jaký kanál byl dominantní nebo také světlost obrázku.

In [ ]:
pixel_counts = np.array(cat.histogram()).reshape((3, 256))

fig, ax = plt.subplots(figsize=(10, 8))
ax.set_title("RGB histogram fotky s kočkou")
ax.bar(range(256), pixel_counts[0, :], color="r", label="r", alpha=0.5)
ax.bar(range(256), pixel_counts[1, :], color="g", label="g", alpha=0.5)
ax.bar(range(256), pixel_counts[2, :], color="b", label="b", alpha=0.5)
ax.legend()
ax.set_xlabel("Počet")
ax.set_ylabel("Hodnota");

Podle histogramu lze usoudit, že se v obrázku vyskytují převážně červené odstíny a to díky ostrému vrcholu červené (`r`) barvy. Zelený (`g`) a modrý (`b`) kanál mají vrcholy při menších hodnotách intenzity než červený kanál.

### 📊 Vliv kalibrací na histogram
Pojďme se společně podívat na to, jaký efekt budou mít změny v jasu, kontrastu, sytosti nebo ostrosti na histogram obrázku.

In [ ]:
def enhance(color, contrast, brightness, sharpness):
    """Creates a view where user can change PIL.ImageEnhance interactively."""
    image = ImageEnhance.Sharpness(
        ImageEnhance.Brightness(
            ImageEnhance.Contrast(ImageEnhance.Color(cat).enhance(color)).enhance(
                contrast
            )
        ).enhance(brightness)
    ).enhance(sharpness)

    img_byte_arr = helpers.get_image_bytes(cat)
    img_byte_arr2 = helpers.get_image_bytes(image)
    target_size = tuple(map(lambda x: x // 4, image.size))
    layout = widgets.HBox(
        [
            widgets.VBox(
                [
                    widgets.HBox(
                        [
                            widgets.Image(
                                value=helpers.get_hist(
                                    cat.resize(target_size), "Original"
                                ),
                            ),
                            widgets.Image(
                                value=helpers.get_hist(
                                    image.resize(target_size), "Processed"
                                ),
                            ),
                        ],
                    ),
                    widgets.HBox(
                        [
                            widgets.Image(
                                value=img_byte_arr,
                            ),
                            widgets.Image(
                                value=img_byte_arr2,
                            ),
                        ],
                        layout=widgets.Layout(height="15%"),
                    ),
                ],
                layout=widgets.Layout(
                    display="flex",
                    flex_flow="column",
                    align_items="center",
                ),
            ),
        ],
        layout=widgets.Layout(
            display="flex",
            flex_flow="column",
        ),
    )

    display(layout)

In [ ]:
# fmt: off
widgets.interact(
    enhance,
    color=widgets.FloatSlider(value=1, min=0, max=1, step=0.2),
    contrast=widgets.FloatSlider(value=1, min=0, max=1, step=0.2),
    brightness=widgets.FloatSlider(value=1, min=0, max=1, step=0.2),
    sharpness=widgets.FloatSlider(value=1, min=0, max=2, step=0.2),
);

### 👯 Histogram matching

Porovnání histogramů je disciplína, ve které **upravíme kontrast jednoho obrázku na základě druhého**. Pojďme se společně podívat, jak bude vypadat náš obrázek kočky ekvalizovaný na základě obrázků z modulu [`scikit-image`](https://scikit-image.org/docs/stable/api/skimage.data.html).

☝️ Pozn.: **Ekvalizace histogramu** je algoritmus, který změní rozložení intenzit v obraze tak, aby se v něm vyskytovaly pokud možno intenzity v širokém rozmezí, a to přibližně se stejnou četností. Pokud ekvalizujeme obrázek jiným obrázkem, snažíme se histogram zdrojového obrázku přiblížit histogramu cílového obrázku.

In [ ]:
def wrap_with_hist(img, label):
    """Returns a VBox with image and its histogram."""
    return widgets.VBox(
        [
            widgets.Image(value=helpers.get_image_bytes(img), height="70%"),
            widgets.Image(value=helpers.get_hist(img, label), height="30%"),
        ],
        layout=widgets.Layout(width="33%"),
    )

In [ ]:
def equalize_image(image):
    """Returns a view with original image, target, and equalized original. With histograms."""
    cat_arr = np.array(cat)
    target = getattr(skimage.data, image)()
    matched = skimage.exposure.match_histograms(cat_arr, target, channel_axis=-1)
    display(
        widgets.Box(
            [
                wrap_with_hist(cat, "Originální"),
                wrap_with_hist(Image.fromarray(target), "Cílový"),
                wrap_with_hist(Image.fromarray(matched), "Ekvalizovaný"),
            ],
            layout=widgets.Layout(
                display="flex",
                flex_flow="row",
                align_items="center",
            ),
        )
    )

In [ ]:
images = []
for image in dir(skimage.data):
    try:
        i = Image.fromarray(getattr(skimage.data, image)())
        if i.mode == "RGB":
            images.append(image)
    except Exception:
        continue

In [ ]:
# fmt: off
widgets.interact(equalize_image, image=images);

## 🏗️ Afinní transformace

**Afinní transformace** nám umožňují transformovat obrázky pomocí lineární algebry 🔠. Kromě klasické lineární transformace zde navíc dochází k přidání vektoru, který nám umožní posunout obrázek v daném směru od počátku ↗️. Jsou vyjádřeny vztahem $P' = P A$, kde $P$ je bod, který transformujeme pomocí matice $A$. Platí $P' = [x' y' w'] = P A = [x y w] A$. Matice $A$ reprezentuje jednotlivé transformace, které mohou být i skládány.

Geometrické afinní transformace souřadnic nám tak umožňují bod $P$ posunout, otočit nebo změnit jeho měřítko (týká se objektů, z těchto bodů složených). ☝️ Pozn.: Bod $\overrightarrow{P}=(x,y)^T$ reprezentujeme třemi homogenními souřadnicemi –> Každý bod v rovině $\overrightarrow{P} \in \mathbb{R^2}$ je reprezentován přímkou $(X,Y,W)$ v $\mathbb{R^3}$. Pozn. 2: Body, kde $W = 0$, se nazývají body v nekonečnu. A bod $[0,0,0]^T$ není povolen.

Více o afinních transformací lze zjistit např. [zde](http://petr.olsak.net/bilin/afinita4.pdf). Pro další úpravy budeme využívat balíček [SymPy](https://www.sympy.org/en/index.html) 🐍, který je určený pro symbolické operace.

In [ ]:
#let's define symbols for symbolic computations
from sympy.functions.elementary import trigonometric as tr

x, y, d_x, d_y, s_x, s_y, theta, c_x, c_y = sympy.symbols(
    "x y d_x d_y s_x s_y theta c_x c_y"
)

Zmiňme základní 2D operace a jejich maticovou reprezentaci:

### 🗜 Škálování
Tzn. změna měřítka (angl. scale, zoom). Určuje poměr mezi aktuální velikostí a cílovou velikostí. Měřítko měníme ve směru osy x, resp. y. $s_x$ určuje škálovací faktor osy x, $s_y$ určuje škálovací faktor osy y. Matice škálování vypadá následovně:

In [ ]:
scale = sympy.Matrix([[s_x, 0, 0], [0, s_y, 0], [0, 0, 1]])
scale

### 😵‍💫 Rotace
Vůči počátku soustavy souřadnic [0,0] parametrem $\theta$ (orientovaný úhel). Matice rotace vypadá následovně:

In [ ]:
rot = sympy.Matrix(
    [[tr.cos(theta), tr.sin(theta), 0], [-tr.sin(theta), tr.cos(theta), 0], [0, 0, 1]]
)
rot

### 🏎 Posunutí
Transformace posunutí, nebo-li translace, probíhá posunem po osách x a y, kdy pro posunutí o $\overrightarrow{d}=(d_x,d_y)^T$ násobíme bod $\overrightarrow{P}=[x,y,1]^T$ maticí:

In [ ]:
tran = sympy.Matrix([[1, 0, d_x], [0, 1, d_y], [0, 0, 1]])
tran

### ✨ Skládání transformací
Při aplikování více transformací na bod $P$ **záleží na pořadí**, v jakém se transformace provádějí. Je rozdíl, jestliže bod posuneme a poté otočíme okolo počátku souřadnicového systému, nebo zda bod nejprve otočíme a poté provedeme transformaci posunutí. 

☝️ Protože záleží na pořadí transformací, záleží i na pořadí násobení matic. Jelikož používáme zápis $P'=PA$, musíme matice reprezentující jednotlivé transformace **násobit zprava**.

Složením výše zmíněných operací

In [ ]:
center = sympy.Matrix([[1, 0, c_x], [0, 1, c_y], [0, 0, 1]])
decenter = sympy.Matrix([[1, 0, -c_x], [0, 1, -c_y], [0, 0, 1]])

sympy.MatMul(tran, scale, decenter, rot, center)

dostaneme pomocí maticového násobení jedinou matici, do které už stačí jen dosadit:

In [ ]:
T = tran @ scale @ decenter @ rot @ center
# T = tran@scale@rot
display(T)

> 🧠❓ Otázka: Proč myslíte, že se zde nacházejí matice $\left[\begin{matrix}1 & 0 & c_{x}\\0 & 1 & c_{y}\\0 & 0 & 1\end{matrix}\right]$ a $\left[\begin{matrix}1 & 0 & -c_{x}\\0 & 1 & -c_{y}\\0 & 0 & 1\end{matrix}\right]$ ?

👀 Napadají vás ještě další operace, o kterých jsme si neřekli?

### ⚙️ Pojďme si to vyzkoušet

V následující buňce je funkce, která vezme zadané parametry, dosadí je do matice a provede s obrázkem danou transformaci. O buňku dále je interaktivní buňka, ve které můžete měnit jednotlivé parametry a sledovat, jak to ovlivní podobu obrázku a podobu matice.

Pozn.: Obrázek bohužel není umístěn na středu canvasu 🙈, a tak je zobrazena pouze jeho část. Je to "feature" knihovny Pillow, kdybyste přišli na zlepšení, založte prosím [issue](https://gitlab.fit.cvut.cz/BI-VIZ/bi-viz/-/issues).

In [ ]:
def affine_transform(
    x_move: int, y_move: int, scale_x: float, scale_y: float, rotation: int
):
    """Performs an affine transformation on image."""
    rotation = np.deg2rad(rotation)
    transformation = (
        T.subs(c_x, -cat.width // 2)
        .subs(c_y, -cat.height // 2)
        .subs(d_x, cat.width // 2)
        .subs(d_y, cat.height // 2)
        .subs(theta, rotation)
        .subs(s_x, scale_x)
        .subs(s_y, scale_y)
        .subs(x, x_move)
        .subs(y, y_move)
    )
    display(widgets.HTMLMath(f"${sympy.latex(transformation)}$"))
    transformation = np.array(transformation.tolist()).astype(np.float64)
    coeff = transformation.flatten()[:6]
    transformed = ImageTransform.AffineTransform(coeff).transform(cat.size, cat)
    display(widgets.Image(value=helpers.get_image_bytes(transformed)))

In [ ]:
# fmt: off
widgets.interact(
    affine_transform,
    x_move=widgets.IntText(value=0),
    y_move=widgets.IntText(value=0),
    scale_x=widgets.FloatText(value=1), # 1 odpovida 100%
    scale_y=widgets.FloatText(value=1), # 1 odpovida 100%
    rotation=widgets.IntText(value=0),
);

# 📐 Škálování

I když teď umíme škálovat (tj. převzorkovávat, tj. proces změny měřítka) pomocí matic, v praxi není nezbytně nutné si manuálně vytvářet transformační matice. `PIL`  obsahuje rozhraní pro škálování obrázků, navíc vám umožní si vybrat převzorkovací metodu, pomocí které se budou dopočítávat hodnoty mezilehlých pixelů po škálování.

☝️ Pozn.: Převzorkování (angl. resampling) mění počet pixelů v obrázku. 

In [ ]:
resamplers = [
    "BICUBIC",
    "BILINEAR",
    "BOX",
    "HAMMING",
    "LANCZOS",
    "NEAREST",
]

Pokud by vás zajímalo, jak jednotlivé metody fungují a jaký je mezi nimi rozdíl, podívejte se na článek na [🌍 Wikipedii](https://en.wikipedia.org/wiki/Image_scaling). 

👀 My si ukážeme, jak vypadají výsledné obrázky. K tomu použijeme `image.resize(NEW_SIZE, resample=RESAMPLER)`:

In [ ]:
def get_scaled_cat(scale):
    """Returns a view with rescaled and resampled images."""
    images = []
    width, height = cat.size

    buffer = [
        widgets.VBox(
            [
                widgets.HTML(f"<center><h4>ORIGINAL</h4><center>"),
                widgets.Image(value=helpers.get_image_bytes(cat)),
            ],
            layout=widgets.Layout(width=f"100%"),
        )
    ]

    for resampler in resamplers:
        if len(buffer) >= 4:
            images.append(widgets.HBox(buffer))
            buffer.clear()

        resampled_image = cat.resize(
            (int(width / scale), int(height / scale)),
            resample=getattr(Image.Resampling, resampler),
        )

        img_byte_arr = helpers.get_image_bytes(resampled_image)

        buffer.append(
            widgets.VBox(
                [
                    widgets.HTML(f"<center><h4>{resampler}</h4><center>"),
                    widgets.Image(value=img_byte_arr),
                ],
                layout=widgets.Layout(width=f"100%"),
            )
        )

    images.append(widgets.HBox(buffer))

    display(
        widgets.VBox(
            images,
            layout=widgets.Layout(display="flex", justify_content="center"),
        )
    )

In [ ]:
# fmt: off
widgets.interact(
    get_scaled_cat, scale=widgets.FloatSlider(min=0.1, max=10, step=0.5, value=3)
);

## [🔁](https://www.youtube.com/watch?v=PGNiXGX2nLU) Rotace a transpozice

`PIL` obsahuje několik předdefinovaných rotací (každých 90˚) a převrácení obrázků. 

☝️ Těchto "transpozicí" můžeme docílit pomocí [`image.transpose(Image.Transpose.TRANSPOSER)`](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.transpose).

In [ ]:
transposers = [
    "FLIP_LEFT_RIGHT",
    "FLIP_TOP_BOTTOM",
    "ROTATE_180",
    "ROTATE_270",
    "ROTATE_90",
    "TRANSPOSE",
    "TRANSVERSE",
]

In [ ]:
images = []
for transposer in transposers:

    if hasattr(Image, 'Transpose'):
        transposed_image = cat.transpose(getattr(Image.Transpose, transposer))
    else: 
        transposed_image = cat.transpose(getattr(Image.Resampling, transposer))

    images.append(
        widgets.VBox(
            [
                widgets.HTML(f"<center><h5>{transposer}</h5><center>"),
                widgets.Image(value=helpers.get_image_bytes(transposed_image)),
            ],
        )
    )

display(
    widgets.HBox(
        images,
        layout=widgets.Layout(display="flex", justify_content="center"),
    )
)

👇 Případně si můžete jakoukoliv rotaci vyrobit vlastnoručně pomocí [`image.rotate(angle=DEGREES)`](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.rotate).

In [ ]:
degrees = 10
total = 0
tmp = cat.copy()


def rotate_cat(cw=True):
    """Creates a view with rotated images."""
    global tmp, degrees, total
    total = 0
    images = []
    for i in range(360 // degrees):
        if cw:
            total -= degrees
        else:
            total += degrees

        images.append(cat.rotate(angle=total))
    image_iterator = (img for img in images)
    size = len(images)
    display(
        widgets.VBox(
            [
                widgets.HBox(
                    [
                        widgets.Image(
                            value=helpers.get_image_bytes(next(image_iterator)),
                            width="25%",
                        )
                        for _ in range(size // 4)
                    ],
                )
                for i in range(4)
            ],
            layout=widgets.Layout(width=f"100%"),
        ),
        layout=widgets.Layout(display="flex", justify_content="center"),
    )

In [ ]:
rotate_cat()

A k čemu je to všechno dobré? Tyto manipulace jsou užitečné například pro [augmentaci dat](https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/) při trénování [konvolučních neuronových sítích (CNN)](https://en.wikipedia.org/wiki/Convolutional_neural_network). 

Představte si, že byste měli sadu portrétových fotografií zvířat, a chtěli byste automaticky říct, jestli je na obrázku kočka 🐈 nebo pes 🐕. 

Pokud bychom měli jen portétové fotky těchto dvou zvířat, budeme schopni na těchto datech natrénovat CNN a s relativně dobrou úspěšností u nich říct, které zvíře je na obrázku. Pokud ale použijeme fotku, na které je náš mazlíček v jiné poloze, může to být pro model celkem obtížná disciplína. Neuronové síti bohužel nemůžeme říct: _"Tak a teď si představ, jak by tahle fotka psa vypadala vzhůru nohama."_ jinak, než že jí takovou fotku vyrobíme. A to neplatí jenom pro `rotace`, ale také pro `škálování`, `zrcadlení` nebo jiné deformace obrazu ( `pokřivení`, `přidání šumu` atd.).

## ✨ Efekty
Kromě transformací obrázků můžeme také měnit jejich barevnou podobu. Některé efekty můžeme aplikovat pomocí [`PIL.ImageOps`](https://pillow.readthedocs.io/en/stable/reference/ImageOps.html#module-PIL.ImageOps). Je to opravdu jen takový malý výčet.


☝️ Například můžeme převést obrázek do [`grayscale`](https://pillow.readthedocs.io/en/stable/reference/ImageOps.html#PIL.ImageOps.grayscale).

In [ ]:
ImageOps.grayscale(cat)

A nebo můžeme černobílý obrázek naopak obarvit pomocí [`PIL.ImageOps.colorize`](https://pillow.readthedocs.io/en/stable/reference/ImageOps.html#PIL.ImageOps.colorize). Specifikujeme RGB barvy pro černé a bílé pixely a metoda vytvoří barevný přechod pro jednotlivé pixely.

☝️ Dále parametrem `mid` můžeme specifikovat třetí barvu pro střed rozsahu pro zajímavější barevný přechod. 

In [ ]:
# https://convertingcolors.com
ImageOps.colorize(
    ImageOps.grayscale(cat),
    mid=(250, 123, 143), # oranžovočervená
    black=(16, 234, 104), # zářivě zelená
    white=(15, 10, 123), # tmavě modrá
)

Nebo můžeme také invertovat barvy obrázku pomocí [`PIL.ImageOps.invert`](https://pillow.readthedocs.io/en/stable/reference/ImageOps.html#PIL.ImageOps.invert).

In [ ]:
ImageOps.invert(cat)

Dalším zajímavým efektem je  [`PIL.ImageOps.solarize`](https://pillow.readthedocs.io/en/stable/reference/ImageOps.html#PIL.ImageOps.solarize), který invertuje jen hodnoty vyšší než je stanovený threshold.

In [ ]:
ImageOps.solarize(cat, threshold=70)

## 🔮 Konvoluce

Dalším způsobem jak manipulovat s obrazovými daty je [konvoluce](https://en.wikipedia.org/wiki/Kernel_(image_processing)). ⚙️ Pro nás se už jedná o **algoritmus pokročilejší**, který přesahuje rámec tohoto kurzu. 🙊 S konvolucí se setkáte během studia ještě několikrát, především u neronových sítí pro obrazová data. My si ji proto představíme pouze vágně a z rychlíku! ☝️ 🧨

Konvoluce je definována jako zobrazení:

$${\displaystyle g(x,y)=\omega *f(x,y)=\sum _{dx=-a}^{a}{\sum _{dy=-b}^{b}{\omega (dx,dy)f(x-dx,y-dy)}},}$$

kde $x$ a $y$ jsou souřadnice "zpracovávaného" pixelu (tj. původního), $f(x, y)$ je funkce vracející hodnotu pixelu, $\omega$ je takzvaný kernel (jádro, reprezentováno maticí) obecně různých, ale často lichých čtvercových rozměrů. Existují také kernely sudých velikostí, s těmi je to ale trochu složitější a nebudeme se jimi zabývat.

Konvoluci si můžete představit tak, že okolo každého pixelu z původního obrázku se udělá výřez o rozměrech kernelu a hodnoty odpovídajících souřadnic těchto dvou matic se mezi sebou vynásobí, pokud to lze. Tyto součiny se následně vysčítají a tento součet je poté použit jako výsledná hodnota nového obrázku.

Pro představu se podívejte na animaci níže 👇

<img src="https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif"/>

K čemu je to dobré? Například se tato metoda používá k extrakci příznaků z obrázků. Pomocí kernelů můžete vyextrahovat hrany objektů z obrázku a můžete tak vašemu modelu dát informaci o tom, že v daném regionu se nachází nějaký objekt a s trochou dalšího úsilí třeba klasifikovat, že je to kočka 🙀.

Tento princip se používá také v [konvolučních neuronových sítích (CNN)](https://en.wikipedia.org/wiki/Convolutional_neural_network), kde každá konvoluční vrstva má nějakou množinu svých kernelů. Jediný rozdíl je ten, že si konvoluční vrstva sama kernely natrénuje a díky tomu, že se v praxi používá několik takových vrstev po sobě, dokáže konvoluční síť detekovat i složitější tvary jako je například kočičí čumák 😺.

Pro více informací o CNN doporučujeme tento [web](https://cs231n.github.io/convolutional-networks/). Během studia se s nimi ještě několikrát setkáte.

🧬 V praxi existuje celá řada konvolučních jader (kernelů). Každé z těchto jader má svůj specifický účel. Pomocí nich jsme schopni nadefinovat mnoho různých operací. Lze tak například definovat **obrazové filtry** (např. pro vyhlazení, doostření apod.) - některé z nich si ukážeme níže 👇 

In [ ]:
def visualize_filter(
    mat: np.ndarray,
    image: Image,
    image2: Image,
    error=None,
    name: str = None,
    attr: dict = None,
):
    """Visualizes origin and processed images, their histograms, kernel (if present) and histogram distance."""
    # get byte arrays
    img_byte_arr = helpers.get_image_bytes(image)
    img_byte_arr2 = helpers.get_image_bytes(image2)

    # matrix latex representation
    matrix = sympy.latex(sympy.Matrix(mat)) if mat is not None else ""

    # If there is a scale and loc for kernel, add it to latex repr.
    if "scale" in attr:
        scale = attr["scale"]
        if scale != 1:
            sgn = scale < 0
            frac = "\\frac{1}{%s}" % abs(scale)
            matrix = f"{'-' if sgn else ''}{frac} {matrix}"
        del attr["scale"]

    if "offset" in attr:
        if attr["offset"]:
            matrix = f'{matrix} + {attr["offset"]}'
        del attr["offset"]
    if "size" in attr:
        del attr["size"]

    layout = widgets.VBox(
        [
            widgets.HTML(f"<h1>{name}</h1>" if name else ""),
            widgets.HTML(
                f'<p> params: {", ".join([f"{k}={v}" for k, v in attr.items()]) if attr else ""} </p>'
                if attr
                else ""
            ),
            widgets.HBox(
                [
                    widgets.VBox(
                        [
                            widgets.HTML("<center><b>Kernel</b><center>"),
                            widgets.HTMLMath(value=f"${matrix}$"),
                            widgets.HTML(
                                "</br><center><b>Histogram Euclidean distance</b><center>"
                            ),
                            widgets.HTML(f"<center>{int(error)}<center>"),
                        ],
                        layout=widgets.Layout(align_items="center", width="250px"),
                    )
                    if mat is not None
                    else widgets.HTML(),
                    widgets.VBox(
                        [
                            widgets.HBox(
                                [
                                    widgets.Image(value=img_byte_arr),
                                    widgets.Image(
                                        value=helpers.get_hist(image, "Original"),
                                    ),
                                ],
                                layout=widgets.Layout(height="300px", width="auto"),
                            ),
                            widgets.HBox(
                                [
                                    widgets.Image(value=img_byte_arr2),
                                    widgets.Image(
                                        value=helpers.get_hist(image2, "Processed"),
                                    ),
                                ],
                                layout=widgets.Layout(height="300px"),
                            ),
                        ],
                        layout=widgets.Layout(),
                    ),
                ],
                layout=widgets.Layout(
                    display="flex",
                    flex_flow="row",
                    align_items="center",
                    padding="5px",
                    width="100%",
                    justify_content="center",
                ),
            ),
        ],
        layout=widgets.Layout(display="flex", flex_flow="column", width="100%"),
    )

    display(layout)

In [ ]:
def apply_image_filter(image_filter: ImageFilter, image: Image):
    """Applies a kernel to image and renders results."""
    kernel = None
    extra = {}
    attrs = list(filter(lambda x: not x[0] == "_" and x != "filter", dir(image_filter)))
    attrs.remove("name")

    if "filterargs" in attrs:
        size, scale, offset, kernel = getattr(image_filter, "filterargs")
        kernel = np.array(kernel).reshape(size)
        attrs.remove("filterargs")
        extra = dict(size=size, scale=scale, offset=offset)

    attr = dict(**{x: getattr(image_filter, x) for x in attrs}, **extra)

    name = image_filter.name
    processed = image.filter(image_filter)
    distance = dist.euclidean(image.histogram(), processed.histogram())
    visualize_filter(kernel, image, processed, error=distance, name=name, attr=attr)

Pro aplikaci filtru použijeme metodu [`image.filter(IMAGE_FILTER)`](https://pillow.readthedocs.io/en/stable/reference/Image.html?highlight=image.filter#PIL.Image.Image.filter), kde `IMAGE_FILTER` je filtr ze submodulu [`PIL.ImageFilter`](https://pillow.readthedocs.io/en/stable/reference/ImageFilter.html?highlight=Imagefilter). 

📖 Pro jednotlivé filtry si můžete pomocí `help(ImageFilter)` zobrazit dokumentaci. Nejvíce vás budou zajímat `filterargs`, ve kterých se dozvíte: velikost kernelu, škálovací faktor, offset a kernel samotný.

In [ ]:
help(ImageFilter.EDGE_ENHANCE)

In [ ]:
# zvyrazneni hran
apply_image_filter(ImageFilter.EDGE_ENHANCE, cat)

In [ ]:
# rozostreni
apply_image_filter(ImageFilter.BLUR, cat)

In [ ]:
# zostreni
apply_image_filter(ImageFilter.SHARPEN, cat)

In [ ]:
# vyhlazeni
apply_image_filter(ImageFilter.SMOOTH, cat)

In [ ]:
# zvyrazneni kontur
apply_image_filter(ImageFilter.CONTOUR, cat)

In [ ]:
# relief
apply_image_filter(ImageFilter.EMBOSS, cat)

In [ ]:
# vytazeni hran
apply_image_filter(ImageFilter.FIND_EDGES, cat)

## 🖌 Sestrojení vlastního kernelu 

In [ ]:
def get_custom_kernel(mat, loc=0, scale=1, name="Custom kernel"):
    """Returns a custom kernel filter based on matrix and loc scale parameters."""
    kernel = ImageFilter.Kernel(mat.shape, mat.flatten(), offset=loc, scale=scale)
    kernel.name = name
    return kernel

In [ ]:
h = np.array([[-1, -1, -1], [2, 2, 2], [-1, -1, -1]])

In [ ]:
apply_image_filter(get_custom_kernel(h, name="Horizontal lines"), cat)

In [ ]:
apply_image_filter(get_custom_kernel(h.T, name="Vertical lines"), cat)

## 🎰 Náhodný kernel

Nebo můžeme zkusit štěstí a vygenerovat si náhodný kernel pomocí generátoru pseudonáhodných čísel.

In [ ]:
kernel_size = np.random.choice([3, 5])
k = np.random.randint(-8, 8, size=(kernel_size, kernel_size))

In [ ]:
apply_image_filter(get_custom_kernel(k, loc=50, scale=-8, name="Random kernel"), cat)

## 📍 Vyhlazovací filtry
V následující sekci si ukážeme filtry, které slouží k **vyhlazování okolí obrázku**. U těchto filtrů se používají čtvercové výběry různých velikostí s nějakou agregační funkci - `mean`, `max`, `min`, `median` atp.  

První filtr `BoxBlur` je průměrem ze čtvercového výběru. Parametr `radius` udává velikost hrany čtverce v jednom směru: Poloměr 0 nerozmazává, vrací identický obrázek. Poloměr 1 zabere 1 pixel v každém směru, tj. celkem 9 pixelů.

In [ ]:
# fmt: off
widgets.interact(
    lambda radius: apply_image_filter(ImageFilter.BoxBlur(radius=radius), cat),
    radius=widgets.IntSlider(val=2, step=1, min=0, max=50),
);

Dalším filtrem je `Gaussian Blur`. Tento filtr nám umožní vyhladit výběr pomocí gaussovské funkce.

Představme si to tak, že máme 2D gaussovskou funkci s předpisem 

$$ G(x, y) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{x^2+y^2}{2\sigma^2}} $$

a grafem funkce

<img src="https://homepages.inf.ed.ac.uk/rbf/HIPR2/figs/gauss2.gif"/>

Pokud bychom tuto funkci zdiskretizovali a přeškálovali, dostali bychom následující matici:

<img src="https://homepages.inf.ed.ac.uk/rbf/HIPR2/figs/gausmask.gif"/>

což můžeme interpretovat tak, že největší váhu bude mít střed kernelu a čím jsou pixely dál od středu, tím menší mají váhu.

In [ ]:
# fmt: off
widgets.interact(
    lambda radius: apply_image_filter(ImageFilter.GaussianBlur(radius=radius), cat),
    radius=widgets.IntSlider(val=2, step=1, min=1, max=50),
);

🌪 Pak už zde máme filtry, které vezmou hodnoty z výběru a aplikují nějakou ze zmíněných agregačních funkcí:

In [ ]:
# vybira nejvyssi hodnotu pixelu v okne o dane velikosti 
# size = velikost kernelu v pixelech 

# fmt: off
widgets.interact(
    lambda size: apply_image_filter(ImageFilter.MaxFilter(size=size), cat),
    size=widgets.Dropdown(options=[x for x in range(1, 30) if x % 2 != 0], value=13),
);

In [ ]:
# vybira median hodnot pixelu v okne o dane velikosti 

# fmt: off
widgets.interact(
    lambda size: apply_image_filter(ImageFilter.MedianFilter(size=size), cat),
    size=widgets.Dropdown(options=[x for x in range(1, 30) if x % 2 != 0], value=13),
);

In [ ]:
# vybira nejnizsi hodnotu pixelu v okne o dane velikosti 

# fmt: off
widgets.interact(
    lambda size: apply_image_filter(ImageFilter.MinFilter(size=size), cat),
    size=widgets.Dropdown(options=[x for x in range(1, 30) if x % 2 != 0], value=13),
);

In [ ]:
# vybira modus hodnot pixelu v okne o dane velikosti 

# fmt: off
widgets.interact(
    lambda size: apply_image_filter(ImageFilter.ModeFilter(size=size), cat),
    size=widgets.Dropdown(options=[x for x in range(1, 30) if x % 2 != 0], value=13),
);

## 🗃️ Segmentace obrazu

Segmentace obrazu je disciplína, v rámci které se snažíme **rozdělit jednotlivé části obrázku** do disjunktních skupin na základě barevných hodnot pixelů, jejich 2D souřadnic a dalších příznaků, které si vyrobíme například pomocí kernelů zmíněných výše. Segmentace obrazu je důležitá - díky ní můžeme detekovat například kde přesně se nachází problém na medicínských snímcích, kde leží předmět, který má zvednout robotická ruka nebo jaký tvar má objekt, který zachycuje bezpečnostní kamera. 

🪚 Jednou (nesupervizovanou) metodou je segmentace pomocí [`K-means algoritmu`](https://en.wikipedia.org/wiki/K-means_clustering) (vizte BI-ML1), který pro předem daný počet shluků rozdělí pixely do skupin podle barev a souřadnic. Jednotlivé shluky pak můžeme použít k "rozřezání" obrázku na podmnožiny, které nás zajímají pro další práci.

Pojďme se podívat, jak to vypadá v praxi. 🕵🏻‍♂️ Pro náš obrázek zkusíme provést segmentaci na `2`, `3`, `4`, ..., až `9` shluků.

🗒️ Pozn.: Kvůli lepší vizualizaci nastavíme každému pixelu ve shluku barvu, která odpovídá průměrné barvě v daném shluku.

In [ ]:
img = np.array(cat)
segmented_images = (helpers.segment_image(n, img) for n in range(2, 10))
clusters = (x for x in range(2, 10))
imgs = []
buffer = []
for im in segmented_images:
    if len(buffer) == 4:
        imgs.append(widgets.HBox(buffer))
        buffer.clear()

    img_byte_arr = helpers.get_image_bytes(im)

    buffer.append(
        widgets.VBox(
            [
                widgets.HTML(f"<center><h4>K={next(clusters)}</h4></center>"),
                widgets.Image(value=img_byte_arr),
            ],
            layout=widgets.Layout(width=f"100%"),
        )
    )

imgs.append(widgets.HBox(buffer))

display(
    widgets.VBox(
        imgs,
        layout=widgets.Layout(display="flex", justify_content="center"),
    )
)

## 🪞Podobnost obrázků na základě histogramu

Jelikož histogram je vlastně n-dimenzionální vektor, můžeme pomocí metrik měřit vzdálenosti mezi jednotlivými obrázky. Pojďme se podívat, které obrázky ze složky `./cats` jsou našemu obrázku kočky nejpodobnější.

In [ ]:
# supported metrics
distance_metrics = [
    "braycurtis",
    "canberra",
    "chebyshev",
    "cityblock",
    "correlation",
    "cosine",
    "dice",
    "euclidean",
    "hamming",
    "jaccard",
    "jensenshannon",
    "kulsinski",
    "minkowski",
    "rogerstanimoto",
    "russellrao",
    "sokalmichener",
    "sokalsneath",
    "sqeuclidean",
    "yule",
]

In [ ]:
cat_images = [Image.open(path) for path in glob.glob("./cats/*")]
cat_histograms = np.vstack(
    [np.array(image.convert("RGBA").histogram()) for image in cat_images]
)
cat_hist = np.array(cat.convert("RGBA").histogram())

In [ ]:
# fmt: off
def get_nearest_images(metric, n):
    distances = np.array(
        [getattr(dist, metric)(cat_hist, cat_histograms[i, :]) for i in range(cat_histograms.shape[0])]
    )
    indices = np.argsort(distances)[:n]

    components = [
        widgets.VBox(
            [
                widgets.HTML(f"<center><h5>Original<h5></center>"),
                widgets.Image(value=helpers.get_image_bytes(cat)),
            ], layout=widgets.Layout(width=f'{100//(n+1)}%')
        )
    ]

    for i in indices.tolist():
        components.append(
         widgets.VBox(
            [
                widgets.HTML(f"<center><h5>i={i}, distance={distances[i]:.4f}<h5></center>"),
                widgets.Image(value=helpers.get_image_bytes(cat_images[i])),
            ],layout=widgets.Layout(width=f'{100//(n+1)}%')
        )
        )
    display(widgets.HBox([
        *components
    ]));

In [ ]:
# fmt: off
widgets.interact(get_nearest_images, metric=distance_metrics, n=widgets.IntSlider(min=1, max=8, value=3));

## 🦄 Shlukování na základě histogramů

A teď, když víme, že můžeme měřit vzdálenost mezi obrázky, nabízí se vyzkoušet si shlukovou analýzu pomocí histogramů těchto obrázků. Např. pomocí [hierarchického shlukování](https://en.wikipedia.org/wiki/Hierarchical_clustering) (znáte z BI-ML1, [dokumentace ve scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html), jehož výsledek vizualizujeme dendogramem. 🌳

In [ ]:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    """Merges a clusters based on linkage method."""
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
def generate_dendogram(p=3, metric="euclid"):
    """Creates a dendogram plot."""
    data = cat_histograms    
    model = AgglomerativeClustering(
        distance_threshold=0,
        n_clusters=None,
        affinity=metric,
        linkage="average",
    )
    out = model.fit_predict(data)
    fig, ax = plt.subplots(figsize=(14, 8))
    ax.set_title("Hierarchical Clustering Dendrogram")

    plot_dendrogram(model, truncate_mode="lastp", p=p)
    ax.set_xlabel("Number of images in node (or index of image if no parenthesis)")
    ax.set_ylabel("Measure of closeness of either individual data points or clusters")

In [ ]:
widgets.interactive(
    generate_dendogram,    
    p=widgets.IntSlider(min=3, max=25, value=25), # počet clusterů
    metric=[
        "cosine",
        "euclidean",
        "l1",
        "l2",
        "manhattan",
    ],
)

☝️ V ukázce výše si můžete všimnout, že volba jiné metriky může kompletně změnit výsledek.

Pomocí buněk níže můžete prozkoumat, jak jsou si jednotlivé blízké obrázky vizuálně podobné.

In [ ]:
indexes = [17, 20, ] # sem zadejte indexy z blízkých obrázků z dendogramu

In [ ]:
display(
    widgets.HBox(
        [
            widgets.Image(
                value=helpers.get_image_bytes(cat_images[i]),
                width=f"{1000//len(indexes)}",
            )
            for i in indexes
        ],
        layout=widgets.Layout(display="flex", justify_content="center"),
    )
)

# 🎉 A to je pro dnešek všechno! 🎉